In [4]:
#!git clone https://github.com/ArthurSSS9966/SoftHebb.git
#%cd SoftHebb

import os.path
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim

# Test to load the SoftHebb package
from dataset import make_data_loaders
from model import load_layers, HebbianOptimizer, AggregateOptim, save_layers
from engine import train_sup, evaluate_sup
from train import check_dimension, training_config
from utils import seed_init_fn, load_presets, load_config_dataset, CustomStepLR
from log import Log

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

params = {"preset":"2SoftMlpMNIST", "dataset_sup":"MNIST", "dataset_unsup":"MNIST",
          "seed":52,"model-name":"2SoftMlpMNIST", "training_mode":"simultaneous", "training_blocks":None,
          "resume":False, "save":False}

name_model = params["preset"]
blocks = load_presets(params["preset"])
CNN_blocks = load_presets("2SoftHebbCnnCIFAR") # Disregard the CIFAR, it is there for the name of the preset

dataset_sup_config = load_config_dataset(params["dataset_sup"], 0.8)
dataset_unsup_config = load_config_dataset(params["dataset_unsup"], 0.8)
dataset_sup_config['validation'] = True

blocks = check_dimension(blocks, dataset_sup_config)
CNN_blocks = check_dimension(CNN_blocks, dataset_sup_config)

train_config = training_config(blocks, dataset_sup_config, dataset_unsup_config, params["training_mode"],
                               params["training_blocks"])

cnn_train_config = training_config(CNN_blocks, dataset_sup_config, dataset_unsup_config, params["training_mode"],
                                   params["training_blocks"])

config = train_config['t1']

train_loader, val_loader, test_loader = make_data_loaders(dataset_sup_config, config['batch_size'], device)


range = 2.165063509461097
range = 1.7320508075688772
block 0, size : 100 14 14
range = 5.0
range = 0.12371791482634838


D:\BlcRepo\OtherCode\NeuroAI\SoftHebb\dataset.py:698: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = torch.tensor(self.data, dtype=torch.float, device=device).div_(255).unsqueeze(1)
D:\BlcRepo\OtherCode\NeuroAI\SoftHebb\dataset.py:700: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.targets = torch.tensor(self.targets, device=device)


In [6]:
SoftHebb_model = load_layers(blocks, name_model, False)
SoftHebb_CNN_model = load_layers(CNN_blocks, "2SoftHebbCnnCIFAR", False)


 Model 2SoftMlpMNIST not found



 ----- Architecture Block FlattenH7841002, number 0 -----
- Flatten(start_dim=1, end_dim=-1)
- HebbSoftLinear(in_features=784, out_features=100, lebesgue_p=2,  bias=False, t_invert=12.0, bias=False, lr_bias=0.0833)
- SoftMax(t_invert=5.0, dim=1)

 ----- Architecture Block Linear(in_, number 1 -----
- Linear(in_features=100, out_features=10, bias=True)

 Model 2SoftHebbCnnCIFAR not found



 ----- Architecture Block BatchNorm2dS11002(5, 5)1.0reflect, number 0 -----
- BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
- HebbSoftConv2d(1, 100, lebesgue_p=2, pruning=0, kernel_size=(5, 5), bias=False, padding_mode=reflect, t_invert=1.0, bias=False, lr_bias=0.1, activation=exp)
- Triangle(power=0.7)
- AvgPool2d(kernel_size=4, stride=2, padding=1)

 ----- Architecture Block FlattenDropout(p=0.5, inplace=False)Linear(in_, number 1 -----
- Flatten(start_dim=1, end_dim=-1)
- Dropout(p=0.5, inplace=False)
- Linear(in_features=19600, 

In [7]:
def train_model(
        final_epoch: int,
        print_freq: int,
        lr: float,
        folder_name: str,
        model,
        device,
        log,
        blocks,
        learning_mode: str = 'BP'
):
    """
    Hybrid training of one model, happens during simultaneous training mode
    """
    print('\n', '********** Hybrid learning of blocks %s **********' % blocks)
    optimizer_sgd = optim.Adam(
        model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    hebbian_optimizer = HebbianOptimizer(model)
    scheduler = CustomStepLR(optimizer_sgd, final_epoch)
    optimizer = AggregateOptim((hebbian_optimizer, optimizer_sgd))
    log_batch = log.new_log_batch()
    
    val_accuracies = []
    storing_path = os.path.join("D:\\BlcRepo\OtherCode\\NeuroAI\\SoftHebb\\Models", folder_name)
        
    for epoch in range(1, final_epoch + 1):
        measures, lr = train_sup(model, criterion, optimizer, train_loader, device, log_batch, learning_mode, blocks)

        if scheduler is not None:
            scheduler.step()

        if epoch % print_freq == 0 or epoch == final_epoch or epoch == 1:

            loss_test, acc_test = evaluate_sup(model, criterion, val_loader, device)
            
            log_batch = log.step(epoch, log_batch, loss_test, acc_test, lr, save=True)
            log.verbose()
            
            print(
                f'Loss test: {loss_test:.4f}, Accuracy test: {acc_test:.2f}%'
            )
            
            val_accuracies.append(acc_test)
        
    return log_batch, val_accuracies

In [8]:
log = Log(train_config)
config = train_config['t1']

SoftHebb_loss, SoftHebb_valloss = train_model(
                                                50,
                                                10,
                                                config['lr'],
                                                name_model,
                                                SoftHebb_model,
                                                device,
                                                log.sup['t1'],
                                                blocks=config['blocks']
                                            )
# Save the model
torch.save({'state_dict': SoftHebb_model.state_dict(),
        'config': SoftHebb_model.config}, os.path.join("D:\\BlcRepo\OtherCode\\NeuroAI\\SoftHebb\\Models", name_model, f"{name_model}.pth"))dw


 ********** Hybrid learning of blocks [0, 1] **********
Epoch: [1/100]	lr: 1.00e-03	time: 00:00:05	Loss_train 0.03194	Acc_train 65.14	/	Loss_test 0.02694	Acc_test 88.12
Loss test: 0.0269, Accuracy test: 88.12%


KeyboardInterrupt: 

In [ ]:
criterion = nn.CrossEntropyLoss()
loss_test, acc_test = evaluate_sup(SoftHebb_model, criterion, test_loader, device)

print(f'Accuracy of the model on the test images: {acc_test:.2f}%')

In [9]:
log = Log(cnn_train_config)
config = cnn_train_config['t1']

CnnHebb_loss, CnnHebb_valloss = train_model(
                                            50,
                                            10,
                                            0.001,
                                            "2SoftHebbCnnMNIST",
                                            SoftHebb_CNN_model,
                                            device,
                                            log.sup['t1'],
                                            blocks=config['blocks']
                                        )

# Save the model
torch.save({'state_dict': SoftHebb_CNN_model.state_dict(),
        'config': SoftHebb_CNN_model.config}, os.path.join("D:\\BlcRepo\OtherCode\\NeuroAI\\SoftHebb\\Models", "2SoftHebbCnnMNIST", f"2SoftHebbCnnMNIST.pth"))


 ********** Hybrid learning of blocks [0, 1] **********
Epoch: [1/100]	lr: 1.00e-03	time: 00:00:47	Loss_train 0.02186	Acc_train 84.76	/	Loss_test 0.00838	Acc_test 86.32
Loss test: 0.0084, Accuracy test: 86.32%
Epoch: [10/100]	lr: 1.00e-03	time: 00:07:33	Loss_train 0.01536	Acc_train 88.81	/	Loss_test 0.01542	Acc_test 91.47
Loss test: 0.0154, Accuracy test: 91.47%
Epoch: [20/100]	lr: 2.50e-04	time: 00:15:12	Loss_train 0.01405	Acc_train 91.39	/	Loss_test 0.00651	Acc_test 95.23
Loss test: 0.0065, Accuracy test: 95.23%
Epoch: [30/100]	lr: 1.25e-04	time: 00:22:55	Loss_train 0.01142	Acc_train 91.80	/	Loss_test 0.00595	Acc_test 95.28
Loss test: 0.0060, Accuracy test: 95.28%
Epoch: [40/100]	lr: 3.13e-05	time: 00:30:56	Loss_train 0.01086	Acc_train 91.59	/	Loss_test 0.00564	Acc_test 95.41
Loss test: 0.0056, Accuracy test: 95.41%
Epoch: [50/100]	lr: 7.81e-06	time: 00:41:12	Loss_train 0.01049	Acc_train 91.73	/	Loss_test 0.00555	Acc_test 95.40
Loss test: 0.0056, Accuracy test: 95.40%


TypeError: cannot pickle 'generator' object

In [10]:
criterion = nn.CrossEntropyLoss()
loss_test, acc_test = evaluate_sup(SoftHebb_CNN_model, criterion, test_loader, device)

print(f'Accuracy of the model on the test images: {acc_test:.2f}%')

Accuracy of the model on the test images: 95.56%


In [ ]:
## Examples of how to load pre-trained models

# Load the model
from model import MultiLayer

SoftMNIST = torch.load("D:\\BlcRepo\OtherCode\\NeuroAI\\SoftHebb\\Models\\2SoftMlpMNIST\\2SoftMlpMNIST.pth")

SoftMNIST_Model = MultiLayer(SoftMNIST['config'])
SoftMNIST_Model.load_state_dict(SoftMNIST['state_dict'])

SoftMNIST_Model.eval()